In [1]:
# Start writing code here...
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import igraph

In /Users/lacemaker/anaconda3/envs/data_env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /Users/lacemaker/anaconda3/envs/data_env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /Users/lacemaker/anaconda3/envs/data_env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /Users/lacemaker/anaconda3/envs/data_env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor rele

This notebook contains template with example of data usage and loading (from prepared csv files, which were produced from original .xlsx file with the help of pandas)

In [2]:
if not ".." in sys.path:
    sys.path.insert(0, "..")

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
os.listdir("../data")

['.DS_Store',
 'description.txt',
 'input_wagon.xlsx',
 'metrics.csv.gz',
 'orders7.csv.gz',
 'orders8.csv.gz',
 'output_wagon.xlsx',
 'sources.csv.gz',
 'standplaces.csv.gz',
 'wagon.zip',
 'wagon_mode_compat.csv.gz']

In [5]:
orders7 = pd.read_csv("../data/orders7.csv.gz")
orders8 = pd.read_csv("../data/orders8.csv.gz")
wagon_mode_compat = pd.read_csv("../data/wagon_mode_compat.csv.gz") # +
sources = pd.read_csv("../data/sources.csv.gz") # +
standplaces = pd.read_csv("../data/standplaces.csv.gz")
metrics = pd.read_csv("../data/metrics.csv.gz") #+

In [196]:
# load data from file (this gets checked and reloaded because of autoreload magic above)
from utils.abstractions import *

In [197]:
roads_graph, name2vertex = get_roads_graph(metrics, debug=False)

In [198]:
roads_graph.es[0]['info'], roads_graph.es[0]['distance'], roads_graph.es[0]['time']

([(100, 104), (200, 138), (305, 355)], 9, 5)

In [199]:
roads_graph.es['weight'] = roads_graph.es['distance']

In [200]:
# dijkstra_paths = roads_graph.shortest_paths_dijkstra(weights="weight")

In [201]:
# np.min(dijkstra_paths), np.max(dijkstra_paths), np.mean(dijkstra_paths), np.median(dijkstra_paths)
sources[sources.Station==190205]

,Station,Date,Units,Stand,WagonModel


In [202]:
gr = init_stations(sources, name2vertex)


In [203]:
190205
gr[0].vs[0]['info']

{'2070': [(1, 50), (1, 50)],
 '2071': [(1, 50)],
 '2135': [(1, 50)],
 '2062': [(1, 50)],
 '2155': [(1, 50)]}

In [204]:
gr, wagons = get_wagon_graph(wagon_mode_compat)

In [205]:
def create_order_graph(orders, name2vertex):
    columns = [
        "OrderNum",
        "Start",
        "Finish",
        #"StartDate",
        "Dur",
        "MinUnit",
        "MaxUnit",
        "Tariff",
        "NeedWagonModel",
        "ShortagePenalty"
    ]
    vertices = np.unique(np.concatenate([
        orders.Start.values,
        orders.Finish.values
    ]))
    for v in vertices:
        v = str(v)
        if v not in name2vertex:
            index = len(name2vertex)
            name2vertex[v] = index
    graph = Graph()
    graph.add_vertices(len(name2vertex))
    edges = []
    for start, finish in orders[["Start", "Finish"]].values:
        start = str(start)
        finish = str(finish)
        u = name2vertex[start]
        v = name2vertex[finish]
        edges.append((u, v))
    # print(edges)
    graph.add_edges(edges)
    graph['order_num'] = orders["OrderNum"].values
    graph['dur'] = orders["Dur"].values
    graph['min_unit'] = orders["MinUnit"].values
    graph['max_unit'] = orders["MaxUnit"].values
    graph['tariff'] = orders["Tariff"].apply(lambda xs: [int(x) for x in xs.split(":")]).values
    graph["wagon_model"] = orders["NeedWagonModel"].values
    graph["penalty"] = orders["ShortagePenalty"].values
    return graph
    
def build_orders(orders, name2vertex):
    columns = [
        "OrderNum",
        "Start",
        "Finish",
        "StartDate",
        "Dur",
        "MinUnit",
        "MaxUnit",
        "Tariff",
        "NeedWagonModel",
        "ShortagePenalty"
    ]
    graphs = dict()
    for start_date, rows in orders7[columns].groupby("StartDate"):
        graph = create_order_graph(rows, name2vertex)
        graphs[start_date] = graph
    # return start_date, rows
    return graphs

In [206]:
graphs = build_orders(orders7, name2vertex)

In [207]:
env = WorldModelEnv(
    orders=orders7,
    wagon_mode_compat=wagon_mode_compat,
    sources=sources, metrics=metrics)

In [208]:
#gr = env.init_world_graph()
env.world_state.vs[0], env.world_state.vs[1]

(igraph.Vertex(<igraph.Graph object at 0x11fdbc750>, 0, {'info': <utils.abstractions.Station object at 0x11cf4ffd0>}),
 igraph.Vertex(<igraph.Graph object at 0x11fdbc750>, 1, {'info': <utils.abstractions.Station object at 0x1b868fd10>}))

In [209]:
#gr.es[0]['route'].arrivals[1] = (1,2,3)

In [210]:
#gr.es[2]['route'].arrivals

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=7ba67d69-ce72-437b-8bf4-6d3c9b906c4a' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>

In [211]:
len(env.wagons)

4453

In [212]:
env

In [213]:
#orders7[orders7['StartDate']==0]
sources[sources.Date==0].Units.sum()

4453

In [214]:
standplaces.head()

,Station,PricePut,PriceStand
0,286408,200,197
1,284008,200,197
2,286408,200,197
3,284008,200,197
4,294809,200,197


In [215]:
np.sum([i == 1 for i in env.world_state.outdegree()])

98

In [216]:
len(env.world_state.es)

287614

In [217]:
#metrics.merge(metrics, how="inner", left_on="To", right_on="From", suffixes=("start_", "end_"))

In [218]:
index = metrics.index
print(len(index))
ignored_nodes = []
while True:
    target = np.unique(metrics.loc[index, "To"])
    source = np.unique(metrics.loc[index, "From"])
    inedges = np.unique(metrics.loc[index, "From"])
    print("Shapes:", target.shape, source.shape, inedges.shape)
    no_outputs = [t for t in target if np.sum(inedges == t) == 0]
    print(len(no_outputs))
    if len(no_outputs) == 0:
        break
    if len(ignored_nodes) == 0:
        ignored_nodes = no_outputs
    else:
        ignored_nodes = np.unique(np.concatenate([ignored_nodes, no_outputs]))
    index = metrics[~metrics.To.isin(ignored_nodes)].index
    print(len(index))
    # break

939404
Shapes: (563,) (1550,) (1550,)
28
905694
Shapes: (535,) (1550,) (1550,)
0


In [219]:
#metrics.loc[index]no_outputs
len(index)

905694

In [220]:
# standplaces[standplaces.Station.isin(ignored_nodes)]

In [221]:
len(env.wagons)

4453

In [222]:
#env.spawn_wagons(env.world_state, step= 3)

In [223]:
wagon_moves = {0: MoveAction(1, "111")}
len(env.wagons)

4453

In [343]:
def get_actions(env):
    my_actions = dict()
    starts = [e for e in env.available_orders.es]
    starts = sorted(starts, key=lambda x: x['penalty'])
    starts = [(e.source, e.target, e['order_num'], e['wagon_model'], e) for e in starts]
    planned = set()
    for s, e, order, wm, ee in starts:
        sel = env.world_state.es.select(_source=s, _target=e)
        road_sel = env.roads_graph.es.select(_source=s, _target=e)
        if len(sel) == 0 or len(road_sel) == 0:
            continue
        edge = sel[0]
        road_sel = road_sel[0]
        move_cost = np.min([v for k, v in road_sel['info']])
        duration = ee['dur']
        min_unit = ee['min_unit']
        max_unit = ee['max_unit']
        tariff = ee['tariff'][0]
        wms = np.unique([e.target for e in env.wagon_graph.es.select(_source=wm)])
        start_wagons = [env.wagons[i] for i in edge.source_vertex['info'].wagons]
        start_wagons = [sx for sx in start_wagons if sx.wm_index in wms]
        start_wagons = [w for w in start_wagons if w.index not in planned]
        if len(start_wagons) == 0:
            break
        #print(start_wagons)
        
        # penalty = ee['penalty']
        #print(ee)
        if len(start_wagons) < min_unit:
            continue
        wagon_ids = [w.index for w in start_wagons][:max_unit]
        for i in wagon_ids:
            planned.add(i)
            my_actions[i] = (s, e, order, duration, tariff)
    return my_actions
    #break

In [344]:
my_actions = get_actions(env)
my_actions
#for r in env.world_state.es[s, e]['route']:
#    print(r.arrivals)


{3469: (1442, 1458, '302201_812941937003_0707', 13, 53514),
 3477: (1442, 1458, '302201_812941937003_0707', 13, 53514),
 3478: (1442, 1458, '302201_812941937003_0707', 13, 53514),
 3853: (1468, 1480, '303071_814704942105_0707', 14, 25768),
 3854: (1468, 1480, '303071_814704942105_0707', 14, 25768),
 3855: (1468, 1480, '303462_830575942105_0707', 14, 25768),
 3856: (1468, 1480, '303462_830575942105_0707', 14, 25768),
 3857: (1468, 1480, '303462_830576942105_0707', 14, 25768),
 3858: (1468, 1480, '303462_830576942105_0707', 14, 25768)}

In [299]:
# 0. validate (? ignore!)
# 1. add moves, move these wagons from stations to routes, compute tariff
# 2. inc current day
# 3. compute costs for the wagons which are at the stations
# 4. check for the newly arrived wagons: move them to destination stations, mark order as done, add profit




In [296]:
#env.available_orders.es[0]
for wid, (s, e, order_num, dur, tariff) in actions.items():
    

In [297]:
#len(env.world_state.vs[e]['info'].wagons), len(env.world_state.vs[s]['info'].wagons)
#start_wagons

In [266]:
# env.station_graph[0].vs[0]['info']
#dir(env.wagon_graph.vs[wm])  #.successors()

In [230]:
#[(s.name, len(s.wagons)) for s in env.world_state.vs['info']]
# env.wagon_graph.vs[wm].predecessors() #successors()
#dir(env.wagon_graph.vs[wm])

array([ 38,  42,  43,  51,  52,  60,  61,  69,  70, 136, 137, 146, 154,
       155, 163, 164, 247, 252, 253, 254, 259, 260, 261, 266, 267, 268,
       273, 274])

[]

In [257]:
[e.target for e in wagon_graph.vs[38].out_edges()]

[38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 42,
 43,
 51,
 52,
 60,
 61,
 69,
 70,
 136,
 137,
 146,
 154,
 155,
 163,
 164,
 247,
 252,
 253,
 254,
 259,
 260,
 261,
 266,
 267,
 268,
 273,
 274]

In [265]:
[e.target for e in wagon_graph.es if e.source==38]

[42,
 43,
 51,
 52,
 60,
 61,
 69,
 70,
 136,
 137,
 146,
 154,
 155,
 163,
 164,
 247,
 252,
 253,
 254,
 259,
 260,
 261,
 266,
 267,
 268,
 273,
 274]